In [194]:
#import above packages
import pandas as pd
import numpy as np
import folium 
import requests
import json
import time
import seaborn as sns
import matplotlib.pyplot as plt
from math import *
import random
from tqdm import tqdm
from iteration_utilities import deepflatten
import statsmodels.api as sm
import statsmodels.formula.api as smf
import folium

pd.options.display.max_columns = 100

In [164]:
URL = "https://listings.offcampusliving.cornell.edu/api/listingById?listingId="
VALID_LIST = valid_ids_flat

def extract(url, ids): 
    s = requests.Session()
    df = pd.DataFrame()
    for i in tqdm(ids):
        time.sleep(random.uniform(0.05, 0.25))
        r = s.get(f"""https://listings.offcampusliving.cornell.edu/api/listingById?listingId={i}""")
        json = r.json()
        if not json is None: 
            new_df = pd.json_normalize(json)
            df = pd.concat([df, new_df], ignore_index=True)
    return df
    
def find_valid_ids(url, ids): 
    s = requests.Session()
    df = pd.DataFrame()
    valid_ids = []
    for i in tqdm(ids):
        time.sleep(random.uniform(0.05, 0.1))
        r = s.get(f"""https://listings.offcampusliving.cornell.edu/api/listingById?listingId={i}""")
        json = r.json()
        if not json is None: 
            valid_ids.append(i)
    pd.Series(valid_ids).to_csv("data/valid_ids.csv", index=False)
    return valid_ids

In [192]:
valid_ids = pd.read_csv("data/valid_ids_05_02_2023.csv")
df = extract(URL, valid_ids.iloc[:,0].to_list())
df = transform(df)
df.to_parquet("data/raw_scrape_05_02_2023.parquet", index=False)
df.listing_id.to_csv("data/valid_ids_05_02_2023.to_csv", index=False)


100%|██████████████████████████████████████████████████████████████████████████████████| 473/473 [01:55<00:00,  4.09it/s]


In [218]:
df = pd.read_parquet("data/raw_scrape_05_02_2023.parquet")
df = apply_filter(df)
# df = df.reset_index(drop=True)
df

,listing_id,account_id,available_at,available_for,address,unit_number,city,zip,contact_name,contact_email,contact_phone,short_description,long_description,rent_amount,rent_type,pets_allowed,gender_preference,num_bedrooms,num_bathrooms,housing_type,gmap_latitude,gmap_longitude,expires_at,listing_within_city,created_at,safety_ratings,listing_photos,coordinates.lat,coordinates.lng,safety_ratings._overall_safety_rating,safety_ratings._category_scores,safety_ratings._overall_safety_rating_as_percentage,safety_ratings._category_scores_as_percentage,safety_ratings._sprinkler_type,safety_ratings._fire_extinguisher_certificate_expiration,safety_ratings._notification_system,safety_ratings._has_valid_certificate_of_occupancy,safety_ratings._meets_minimum_requirements,safety_ratings._exceeds_requirements,safety_ratings._has_fire_resistant_construction_type,safety_ratings._satisfies_applicable_code,safety_ratings._certificate_expiration_date,num_rooms,amenity_air_conditioning,inclusion_electricity,electronic_payments_allowed,inclusion_furniture,inclusion_heat,inclusion_internet,amenity_kitchen_available,amenity_laundry_available,is_near_bus,amenity_offstreet_parking_available,inclusion_offstreet_parking,psparking_available,inclusion_water,market_grad,market_faculty,market_ugrad,market_sabbatical,market_fall_sublet,market_spring_sublet,market_summer_sublet,market_short_term,distance_to_campus,price_per_room,price_per_unit
0,20,19,2023-08-01 00:00:00+00:00,12,219 Kelvin Place,1,Ithaca,14850,Kathleen Cacciotti,kelvinplaceapartments@gmail.com,6072275031,Furnished Rooms in Large North Campus Area Hou...,Great location with close easy access to Corne...,795.0,Price Per Unit,0,No Preference,1.0,3.0,Room to Rent,42.456514,-76.483689,2023-06-19T00:00:00.000Z,No,2021-11-29 15:19:58.835000+00:00,None,"[{'AddedDateTime': '2017-03-29T14:11:54.000Z',...",42.456514,-76.483689,NaN,None,NaN,None,None,None,None,None,None,None,None,None,None,2.0,0,1,1,1,1,1,1,1,1,1,1,0,1,1,0,1,0,0,0,0,0,991.667090,795.0,795.0
1,23,21,2023-08-01 00:00:00+00:00,12,116 LAKE STREET,,Ithaca,14850,Manager,tterpen@stny.rr.com,(607) 279-8742,One Bedroom Furnished. Very high quality. We...,Ideal Grad Student/Faculty Housing. High Quali...,1590.0,Price Per Unit,0,No Preference,1.0,1.0,Rent,42.450724,-76.492924,2023-04-27T00:00:00.000Z,Yes,2021-11-29 15:19:58.917000+00:00,None,"[{'AddedDateTime': '2019-03-03T16:07:54.000Z',...",42.450724,-76.492924,12.0,"[{'Applicable Code': None, 'Construction Type'...",26.0,"[{'Applicable Code': None, 'Construction Type'...",[],2023-11-15T00:00:00.000Z,[{'notificationSystemCertificateExpiration': '...,True,True,True,False,False,2023-10-12T00:00:00.000Z,2.0,0,0,0,1,1,0,1,1,1,1,0,0,1,1,1,0,1,0,0,0,0,737.144681,1590.0,1590.0
2,39,37,2023-08-05 00:00:00+00:00,12,232 LINDEN AVENUE,None,Ithaca,14850,IKE NESTOPOULOS,bgm900@gmail.com,6073391137,Visit our website demosjohnnycollegetownrental...,Visit our website \ndemosjohnnycollegetownrent...,1050.0,Price Per Person,0,No Preference,5.0,2.0,Rent,42.440776,-76.484404,2022-09-24T00:00:00.000Z,Yes,2021-11-29 15:19:59.363000+00:00,None,"[{'AddedDateTime': '2022-08-24T18:50:00.444Z',...",42.440776,-76.484404,8.0,"[{'Applicable Code': None, 'Construction Type'...",17.0,"[{'Applicable Code': None, 'Construction Type'...",[{'sprinklerCertificateExpiration': '2022-08-0...,2022-05-31T00:00:00.000Z,[{'notificationSystemCertificateExpiration': '...,True,False,False,False,False,2024-08-12T00:00:00.000Z,6.0,0,1,1,1,1,1,1,1,1,1,0,0,1,1,0,1,0,0,0,0,0,765.302946,1050.0,1050.0
3,40,37,2023-06-06 00:00:00+00:00,12,232 LINDEN AVENUE,None,Ithaca,14850,john puglia,bgm900@gmail.com,(631) 839-3590,Visit our website\ndemosjohnnycollegetownrenta...,Visit our website \ndemosjohnnycollegetownrent...,1100.0,Price Per Person,0,No Preference,2.0,2.0,Rent,42.440884,-76.484107,2022-09-24T00:00:00.000Z,Yes,2021-11-29 15:19:59.398000+00:00,None,"[{'AddedDateTime': '2017-03-31T17:56:13.000Z',...",42.440884,-76.484107,8.0,"[{'Applicable Code': N

In [282]:
df = pd.read_parquet("data/raw_scrape_05_02_2023.parquet")
df = apply_filter(df)
df.iloc[7].listing_photos[3]

{'AddedDateTime': '2021-12-02T13:02:33.442Z',
 'IsDefault': False,
 'PhotoId': 25064,
 'PhotoName': 'Hallway.jpg',
 'PhotoUrl': 'https://images.listings.offcampusliving.cornell.edu/listings/124/Hallway.jpg',
 'listingId': 124}

In [ ]:
df = pd.read_parquet("data/raw_scrape_05_02_2023.parquet")
df = apply_filter(df)

photo_urls = []
for i, listing_id in tqdm(enumerate(df.listing_id.unique())):
    for j in range(len(df.iloc[i].listing_photos)):
        if len(df.iloc[i].listing_photos) > 0: 
            time.sleep(random.uniform(0.1, 0.2))
            url = df.iloc[i].listing_photos[j]['PhotoUrl']
            img_data = requests.get(url).content
            with open(f'''photos/{listing_id}_{j}.{url.split(".")[-1]}''', 'wb') as handler:
                handler.write(img_data)


144it [17:20,  2.15s/it]

In [200]:
DROP_COLS = ['Unnamed: 0',
     'ContactPreference', 
     'PaymentStatus',
     'LocationConfirmed',
     'Active',
     'LastUpdated', 
     'ListingComplete',
     'safetyRatingAddress',
     'SafetyRatings.BuildingAddress',
     'SafetyRatings.DateLastUpdated']
 
def load_data(filepath="data/raw_scrape_oct_25_2021.csv"):  
    """
    param filepath: str of filename (including '.csv')
    rtype: pd.DataFrame
    """
    df = pd.read_csv(filepath)
    return df


# Using haversine function, google map data to compute a distance to McGraw Clocktower (a proxy for campus)
# for each of these listings, we want to compute the haversine dist from the listing to the clocktower. 
# haversine code from police allocation activity. 
def haversine_distance(pt1, pt2):
    """
    type pt1: tuple 
    type pt2: tuple
    rtype: float (m)
    """
    lat1,lon1 = pt1
    lat2,lon2 = pt2
    # convert decimal degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    # haversine formula
    a = sin((lat2 - lat1)/2)**2 + cos(lat1) * cos(lat2) * sin((lon2 - lon1)/2)**2
    c = 2 * asin(sqrt(a))
    return float(6371000*c)

# McGraw Clocktower (lat,long): 42.44764589643703, -76.48496728637923
def add_distance_to_campus(df, lat=42.44764589643703, long=-76.48496728637923): 
    """
    Uses haversine_distance to calculate and record distance_to_campus (in meters) for each listing in DataFrame
    """
    distances = []
    for index, row in df.iterrows(): 
        listing_latitude = row["gmap_latitude"]
        listing_longitude = row["gmap_longitude"]
        distances.append((haversine_distance((listing_latitude,listing_longitude),((lat,long)))))
    df["distance_to_campus"] = distances
    df["distance_to_campus"] = df["distance_to_campus"].astype(float)
    return df

def add_price_per_room(df):
    """
    """
    df["rent_amount"] = df["rent_amount"].astype(float)
    df["price_per_room"] = df.apply(lambda row: row.rent_amount/(row.num_bedrooms+1) if row.rent_type == "Price per Unit" else row.rent_amount, axis=1)
    return df 

def add_price_per_unit(df):
    """
    """
    df["price_per_unit"] = df.apply(lambda row: row.rent_amount*(row.num_bedrooms+1) if row.rent_type == "Price per Person" else row.rent_amount, axis=1)
    return df

def convert_to_snake_case(column_name):
    """
    Returns: snake_case version of a given column name
    
    param column_name: 
    type column_name: 
    rtype: str
    """
    return "".join(["_"+value.lower() if value.isupper() else value for value in column_name]).lstrip("_")


def flatten_amenities(df):
    """
    Adds binary columns for every possible amenity, sets to 1 for a given unit if unit includes that amenity
    """
    potential_amenities = {'Air Conditioning' : 'air_conditioning'
                           ,'Electricity Included' : 'electricity_included'
                           ,'Electronic Payments Accepted ': 'electronic_payments'
                           ,'Furnished' : 'furniture_included'
                           ,'Heat Included': 'heat_included'
                           ,'Internet Included' : 'internet_included'
                           ,'Kitchen': 'kitchen_available'
                           ,'Laundry Facilities' : 'laundry_available'
                           ,'Near Bus Route' : 'near_bus'
                           ,'Off-Street Parking Available' : 'osparking_available'
                           ,'Off-Street Parking Included' : 'osparking_included'
                           ,'Permitted Street Parking Available' : 'psparking_available'
                           ,'Water Included' : 'water_included'} 
    amenities_list = [i for i in potential_amenities.keys()]
    for i in amenities_list: 
        df[potential_amenities[i]] = df["amenities"].apply(lambda x: 1 if i in x else 0)    
    return df


def flatten_listing_types(df):
    """
    Adds binary columns for every possible listing type, sets to 1 for a given unit if unit is of that listing type  
    """
    potential_listing_types = {'Graduate' : 'graduate',
                 'Staff/Faculty' : 'faculty',
                 'Undergraduate' : 'ugrad', 
                 'Sabbatic Housing' : 'sabbatical',
                 'Fall Sublet' : 'fa_sublet',
                 'Spring Sublet' : 'sp_sublet', 
                 'Summer Sublet' : 'sum_sublet',
                 'Short-Term Housing' : 'short_term'}
    listing_types_list = [i for i in potential_listing_types.keys()]
    for i in listing_types_list:
        df[potential_listing_types[i]] = df["listing_types"].apply(lambda x: 1 if i in x else 0)
    return df 

def get_column_rename_mapping(): 
    """
    """
    rename_mapping = {"air_conditioning":"amenity_air_conditioning"
                      ,"bathrooms":"num_bathrooms"
                      ,"bedrooms":"num_bedrooms"
                      ,"create_date":"created_at"
                      ,"date_available":"available_at"
                      ,"electronic_payments":"electronic_payments_allowed"
                      ,"faculty":"market_faculty"
                      ,"graduate":"market_graduate"
                      ,"kitchen_available":"amenity_kitchen_available"
                      ,"laundry_available":"amenity_laundry_available"
                      ,"length_available":"available_for"
                      ,"listing_address":"address"
                      ,"listing_city":"city"
                      ,"listing_expiration_date":"expires_at"
                      ,"listing_zip":"zip"
                      ,"near_bus":"is_near_bus"
                      ,"num_rooms":"num_rooms"
                      ,"water_included":"inclusion_water"
                      ,"osparking_available":"amenity_offstreet_parking_available"
                      ,"osparking_included":"inclusion_offstreet_parking"
                      ,"electricity_included":"inclusion_electricity"
                      ,"furniture_included":"inclusion_furniture"
                      ,"heat_included":"inclusion_heat"
                      ,"internet_included":"inclusion_internet"
                      ,"pets":"pets_allowed"
                      ,"price_per_room":"price_per_room"
                      ,"price_per_unit":"price_per_unit"
                      ,"sabbatical":"market_sabbatical"
                      ,"fa_sublet":"market_fall_sublet"
                      ,"sp_sublet":"market_spring_sublet"
                      ,"sum_sublet":"market_summer_sublet"
                      ,"ugrad":"market_ugrad"
                      ,"graduate":"market_grad"
                      ,"short_term":"market_short_term"
                      ,"water_included":"inclusion_water"}
    return rename_mapping 

# Setting datatypes
def set_dtypes(df): 
    """
    Sets the datatype of each column in the given DataFrame to ensure standardization 
    """
    # Floats
    df["price_per_unit"] = df["price_per_unit"].astype(float)
    df["price_per_room"] = df["price_per_room"].astype(float)
#     df["rent_per_person"] = df["rent_per_person"].astype(float)
    df["gmap_latitude"] = df['gmap_latitude'].astype(float)
    df["gmap_longitude"] = df['gmap_longitude'].astype(float)
    df["num_bedrooms"] = df["num_bedrooms"].astype(float)
    df["num_bathrooms"] = df["num_bathrooms"].astype(float)
    # Datetimes
    df["created_at"] =  pd.to_datetime(df["created_at"])
    df["available_at"] =  pd.to_datetime(df["available_at"])
    # Dummy Variables (bools as ints) 
    df["pets_allowed"] = df["pets_allowed"].astype(int)
    df['amenity_air_conditioning'] = df['amenity_air_conditioning'].astype(int)
    df['inclusion_electricity'] = df['inclusion_electricity'].astype(int)
    df['inclusion_furniture'] = df['inclusion_furniture'].astype(int)
    df['inclusion_heat'] = df['inclusion_heat'].astype(int)
    df['inclusion_internet'] = df['inclusion_internet'].astype(int)
    df['amenity_kitchen_available'] = df['amenity_kitchen_available'].astype(int)
    df['amenity_laundry_available'] = df['amenity_laundry_available'].astype(int)
    df['inclusion_water'] = df['inclusion_water'].astype(int)
    df['is_near_bus'] = df['is_near_bus'].astype(int)
    df['amenity_offstreet_parking_available'] = df['amenity_offstreet_parking_available'].astype(int)
    df['inclusion_offstreet_parking'] = df['inclusion_offstreet_parking'].astype(int)
    df['market_faculty'] = df['market_faculty'].astype(int)
    df['market_ugrad'] = df['market_ugrad'].astype(int)
    df['market_sabbatical'] = df['market_sabbatical'].astype(int)
    df['market_fall_sublet'] = df['market_fall_sublet'].astype(int)
    df['market_spring_sublet'] = df['market_spring_sublet'].astype(int)
    df['market_summer_sublet'] = df['market_summer_sublet'].astype(int)
    df['market_short_term'] = df['market_short_term'].astype(int)
    df['market_grad'] = df['market_grad'].astype(int)
    return df 

def remap_to_int(df):
    """
    rtype: DataFrame
    """
    df["pets_allowed"] = df["pets_allowed"].apply(lambda x: 1 if x=="Yes" else 0).astype(int)
    df["num_bedrooms"] = df["num_bedrooms"].apply(lambda x: 0 if x=="studio" else x)
    df["num_bedrooms"] = df["num_bedrooms"].astype(float)
    df["num_rooms"] = df["num_bedrooms"]+1
    return df

def remove_errant_rows(df): 
    """
    Removes errant values from the DataFrame
    """
    # Entries with 'test' unit number (presumably for use by site admins)
    df = df.loc[df.unit_number != 'test']
    # Entries without a listing price
    df = df.loc[df.price_per_room > 10]
    # Entries without valid latitude/longitude
    df = df.loc[df["gmap_latitude"].notna()]
    df = df.loc[df['gmap_longitude'].notna()]
    return df

def remove_implausible_rows(df): 
    """
    Removes implausible values from the DataFrame
    """
    # Entries outside 5 kilo|meter radius from Clocktower
    df = df.loc[df.distance_to_campus <= 5000]
    df = df.loc[df.num_bathrooms >= 0]
    df = df.loc[df.num_bedrooms >= 0]
    df = df.loc[df.num_rooms >= 0]
#     df = df.loc[(df.active == 0) | (df.active == 1)]
#     df = df.loc[(df.amenity_air_conditioning == 0) | (df.amenity_air_conditioning == 1)]
#     df = df.loc[(df.pets_allowed == 0) | (df.pets_allowed == 1)]
#     df = df.loc[(df.inclusion_electricity == 0) | (df.inclusion_electricity == 1)]
#     df = df.loc[(df.inclusion_heat == 0) | (df.inclusion_heat == 1)]
#     df = df.loc[(df.amenity_kitchen_available == 0) | (df.amenity_kitchen_available == 1)]
#     df = df.loc[(df.internet_included == 0) | (df.internet_included == 1)]
#     df = df.loc[(df.laundry_available == 0) | (df.laundry_available == 1)]
#     df = df.loc[(df.near_bus == 0) | (df.near_bus == 1)]
    return df 

def transform(data):
    """
    Transforms a given df 
    
    param data: dataframe to transform
    type data: pandas.Dataframe
    rtype: pandas.Dataframe
    """
    df = (
        data
        .drop(columns = [col for col in data if col in DROP_COLS])
        .rename(mapper=convert_to_snake_case, axis=1)
        .rename(columns=get_column_rename_mapping())
        .pipe(remap_to_int)
        # .set_index('listing_id')
        .pipe(flatten_amenities)
        .pipe(flatten_listing_types)
        .drop(["amenities", "listing_types"],axis=1)
        .pipe(add_distance_to_campus)
        .pipe(add_price_per_room)
        .pipe(add_price_per_unit)
        .rename(columns=get_column_rename_mapping())
        .pipe(set_dtypes)
        .pipe(remove_errant_rows)
        .pipe(remove_implausible_rows)
    )
    return df 


def convert_meters_to_feet(value, unit_from="meters", unit_to="feet", output_type=float, prefix='', factor_power=None):
    """
    A function to convert (prefix)meters to feet of output_type
   
    param value: the measurement in (prefix)meters to convert
    type value: float
    param unit_from (optional): one of "meters", "feet". Defaults to "meters"
    type unit_from: str
    param unit_to (optional): one of "meters", "feet". Defaults to "feet"
    type unit_to: str
    param prefix (optional): a valid International System of Units metric prefix.
        (i.e. "kilo"), see wikipedia.org/wiki/Metric_prefix. Case Insensitive. Defaults to ''
    type prefix: str 
    param output_type (optional): desired output type. Defaults to float
    type output_type: one of str, int, float
    param factor_power (optional): the base 10 exponent of desired prefix. Defaults to None 
    type factor_power: int
 
    rtype: output_type 
    """
    possible_prefixes={"yotta":24,"zetta":21,"exa":18,"peta":15,"tera":12,"giga":9,"mega":6,"kilo":3,"hecto":2,"deca":1,
                       "":0,"deci":-1,"centi":-2,"milli":-3,"micro":-6,"nano":-9,"pico":-12,"femto":-15,"atto":-18,"zepto":-21,"yocto":-24}
    possible_conversion_factors = {"meters_to_feet":3.28, "feet_to_meters":0.3048, "feet_to_feet":1, "meters_to_metters":1}
    conversion_factors_key = unit_from+"_to_"+unit_to
    if factor_power is None:
        factor_power = possible_prefixes[prefix.lower()]
    value_scaled_to_meter = value*(10**factor_power)
    conversion_factor = possible_conversion_factors[conversion_factors_key]
    output = value_scaled_to_meter*conversion_factor
    return output_type(output)


def apply_filter(df): 
    """
    """
    df = df.loc[df.market_spring_sublet == 0]
    df = df.loc[df.market_fall_sublet == 0]
    df = df.loc[df.market_summer_sublet == 0]
    df = df.loc[df.market_short_term == 0]
    df = df.loc[df.num_bedrooms <= 20]
    df = df.loc[df.price_per_unit <= 15000]
    df = df.loc[df.distance_to_campus <= 2500]
    return df 

In [196]:
def generate_map(df=df,gmapLatitude=42.44764589643703,gmapLongitude=-76.48496728637923):
    """
    param: df
    type: pd.DataFrame() Object
    param gmapLatitude: 
    type df.gmapLatitude: float 
    param gmapLongitude: 
    type df.gmapLongitude: float
    rtype: folium.Map() Object
    """
    res = folium.folium.Map(location=[42.44764589643703,-76.48496728637923],width='100%', height='100%', left='0%', top='0%',
                     zoom_start=14,position='relative', tiles='OpenStreetMap')
    folium.vector_layers.CircleMarker([42.44764589643703, -76.48496728637923], radius=2.5, color='red', opacity=1).add_to(res)
    for index, row in df.iterrows(): 
        folium.vector_layers.CircleMarker([row["gmap_latitude"], row["gmap_longitude"]], radius=0.2, color="blue", opacity=0.5).add_to(res)
    folium.vector_layers.Circle(location=[42.44764589643703,-76.48496728637923],radius=convert_meters_to_feet(2.5,prefix="kilo", unit_from="meters",
        unit_to="feet",output_type=float),color="white").add_to(res)
    return res

In [197]:
generate_map(df)

In [ ]:
# import statsmodels.api as sm
# import statsmodels.formula.api as smf

# fit = smf.ols('logTruePrice ~ bedrooms + bathrooms + dist_to_cornell+C(air_conditioning)+C(electricity_included)+C(electronic_payments)+C(furniture_included)' \
#                      '+C(heat_included)+C(internet_included)+C(kitchen_available)+C(laundry_available)+C(near_bus)+C(osparking_available)+C(osparking_included)+C(water_included)',data=regression_vars).fit() 
# fit.summary()